# LimberJack.jl Tutorial

The purpose of this notebook is show the user how to interface ```LimberJack.jl``` with ```Turing.jl``` to perform statistical inference.

## Usage

In [7]:
# The statistical inference frame-work we will use
using Turing
# Our code
using LimberJack
# Some data management libs.
using CSV
using NPZ
using YAML
# Some Lin. Alg.
using LinearAlgebra
# Allows us to use Python when needed
using PythonCall
sacc = pyimport("sacc")

Python module: <module 'sacc' from '/home/jaimerz/.julia/environments/v1.7/.CondaPkg/env/lib/python3.10/site-packages/sacc/__init__.py'>

### Load data

First download the data ```.fits``` file from https://drive.google.com/drive/folders/1gkSCxEVRLGQsBHkDsSgOnmKhSmOuT_BT?usp=sharing and move to the ```data``` folder in the ```LimberJack``` repository.

Now we must convert our data into a format that ```LimberJack.jl``` can parse. 
The main obstacle is the lack of a ```Julia``` implementation of ```sacc.py``` (```yaml.jl``` exists!).
Thankfully we can call ```sacc.py``` from ```Julia```. 
This allows us to load both the relevant ```.yml``` and ```.fits``` files.
These files can then me passed to  ```LimberJack.jl ```'s  ```make_data ``` function to generate the  ```meta ``` structure and the  ```files ``` dictionary composed respectively of:

    + ```meta```
        + ```tracers```: a list of all tracers 
        + ```pairs```: a list of all pairs of tracers for the $C_l$'s
        + ```cls```: the concatenated flat data vector
        + ```idx```: indecies of the data vector corresponding to the first $l$ of each $C_l$ such that ```cli = cls[idx[i]:idx[i+1]]```
        + ```cov```: covariance matrix of the data vector
        + ```inv_cov```: inverse of the former
    + ```files.npz```
        + ```ls_tracer_tracer```: an array containing the $l$'s for the tracer pair
        + ```nz_tracer```: the redshift distribution of galaxies of a given tracer as ```[z, dndz]```. 


In [9]:
sacc_path = "data/cls_FD_covG.fits"
yaml_path = "data/DESY1.yml"
sacc_file = sacc.Sacc().load_fits(sacc_path)
yaml_file = YAML.load_file(yaml_path)
meta, files = make_data(sacc_file, yaml_file)

data_vector = meta.data
cov_tot = meta.cov;

DESgc__0 DESgc__0 5
DESgc__1 DESgc__1 8
DESgc__2 DESgc__2 10
DESgc__3 DESgc__3 11
DESgc__4 DESgc__4 13
DESgc__0 DESwl__0 5
DESgc__0 DESwl__1 5
DESgc__0 DESwl__2 5
DESgc__0 DESwl__3 5
DESgc__1 DESwl__0 8
DESgc__1 DESwl__1 8
DESgc__1 DESwl__2 8
DESgc__1 DESwl__3 8
DESgc__2 DESwl__0 10
DESgc__2 DESwl__1 10
DESgc__2 DESwl__2 10
DESgc__2 DESwl__3 10
DESgc__3 DESwl__0 11
DESgc__3 DESwl__1 11
DESgc__3 DESwl__2 11
DESgc__3 DESwl__3 11
DESgc__4 DESwl__0 13
DESgc__4 DESwl__1 13
DESgc__4 DESwl__2 13
DESgc__4 DESwl__3 13
DESwl__0 DESwl__0 24
DESwl__0 DESwl__1 24
DESwl__0 DESwl__2 24
DESwl__0 DESwl__3 24
DESwl__1 DESwl__1 24
DESwl__1 DESwl__2 24
DESwl__1 DESwl__3 24
DESwl__2 DESwl__2 24
DESwl__2 DESwl__3 24
DESwl__3 DESwl__3 24


### Defining the model

Now it is time to create the ```Turin.jl``` model. This is done through a function using the ```@model``` decorator.
The important detail here is that the model can only be a function of the data (upon which we will condition the model to obtain constraints). The model function must accomplish the following:
1. Define the priors for the parameters. This is done using the syntax of ```Distributions.jl``` for statistical programming. Note that the nuisance parameters must follow a strict naming convention in terms of the tracer they belong to. Otherwise, they will not be recognized. Also in order to pass the nuisance parameters our machinary they must be written as a dictionary. 
2. Initiate an instance of ```cosmo = LimberJack.Cosmology``` which will generate the matter power spectrum for the sampled cosmological parameters upon initializing. 
3. Evaluate ```LimberJack.Theory(cosmo, tracers, pairs, idx, files; Nuisances=nuisances)``` which for a given instance of  ```LimberJack.Cosmology``` it will return the concatenated $C_l$'s vector for the considered tracers, tracers' pairs, their indeces in the data vector and their auxialiry files  ($l$'s and $n(z)$'s).

In [11]:
@model function model(data; #data dependence!!!
                      files=files,
                      meta=meta,
                      cov=cov_tot) 
    # Define priors
    #KiDS priors
    Ωm ~ Uniform(0.2, 0.6)
    Ωb ~ Uniform(0.028, 0.065)
    h ~ TruncatedNormal(72, 5, 0.64, 0.82)
    s8 ~ Uniform(0.4, 1.2)
    ns ~ Uniform(0.84, 1.1)

    DESgc__0_b ~ Uniform(0.8, 3.0)
    DESgc__1_b ~ Uniform(0.8, 3.0)
    DESgc__2_b ~ Uniform(0.8, 3.0)
    DESgc__3_b ~ Uniform(0.8, 3.0)
    DESgc__4_b ~ Uniform(0.8, 3.0)
    DESgc__0_dz ~ TruncatedNormal(0.0, 0.007, -0.2, 0.2)
    DESgc__1_dz ~ TruncatedNormal(0.0, 0.007, -0.2, 0.2)
    DESgc__2_dz ~ TruncatedNormal(0.0, 0.006, -0.2, 0.2)
    DESgc__3_dz ~ TruncatedNormal(0.0, 0.01, -0.2, 0.2)
    DESgc__4_dz ~ TruncatedNormal(0.0, 0.01, -0.2, 0.2)
    DESwl__0_dz ~ TruncatedNormal(-0.001, 0.016, -0.2, 0.2)
    DESwl__1_dz ~ TruncatedNormal(-0.019, 0.013, -0.2, 0.2)
    DESwl__2_dz ~ TruncatedNormal(0.009, 0.011, -0.2, 0.2)
    DESwl__3_dz ~ TruncatedNormal(-0.018, 0.022, -0.2, 0.2)
    DESwl__0_m ~ Normal(0.012, 0.023)
    DESwl__1_m ~ Normal(0.012, 0.023)
    DESwl__2_m ~ Normal(0.012, 0.023)
    DESwl__3_m ~ Normal(0.012, 0.023)
    A_IA ~ Uniform(-5, 5) 
    alpha_IA ~ Uniform(-5, 5)

    nuisances = Dict("DESgc__0_b" => DESgc__0_b,
                     "DESgc__1_b" => DESgc__1_b,
                     "DESgc__2_b" => DESgc__2_b,
                     "DESgc__3_b" => DESgc__3_b,
                     "DESgc__4_b" => DESgc__4_b,
                     "DESgc__0_dz" => DESgc__0_dz,
                     "DESgc__1_dz" => DESgc__1_dz,
                     "DESgc__2_dz" => DESgc__2_dz,
                     "DESgc__3_dz" => DESgc__3_dz,
                     "DESgc__4_dz" => DESgc__4_dz,

                     "A_IA" => A_IA,
                     "alpha_IA" => alpha_IA,

                     "DESwl__0_dz" => DESwl__0_dz,
                     "DESwl__1_dz" => DESwl__1_dz,
                     "DESwl__2_dz" => DESwl__2_dz,
                     "DESwl__3_dz" => DESwl__3_dz,
                     "DESwl__0_m" => DESwl__0_m,
                     "DESwl__1_m" => DESwl__1_m,
                     "DESwl__2_m" => DESwl__2_m,
                     "DESwl__3_m" => DESwl__3_m)

    cosmology = Cosmology(Ωm, Ωb, h, ns, s8,
                          tk_mode="EisHu",
                          Pk_mode="Halofit")

    theory = Theory(cosmology, meta, files; Nuisances=nuisances)
    data ~ MvNormal(theory, cov)
end;

Now it is time to condition the model on the data. This is simply done as:

In [15]:
iterations = 10
adaptation = 10
TAP = 0.65

stat_model = model(data_vector)
sampler = NUTS(adaptation, TAP)

NUTS{Turing.Essential.ForwardDiffAD{0}, (), AdvancedHMC.DiagEuclideanMetric}(10, 0.65, 10, 1000.0, 0.0)

### Sampling

In [16]:
chain = sample(stat_model, sampler, iterations;
               progress=true, save_state=true)

┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC ~/.julia/packages/AdvancedHMC/eLdgX/src/hamiltonian.jl:49
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC ~/.julia/packages/AdvancedHMC/eLdgX/src/hamiltonian.jl:49
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC ~/.julia/packages/AdvancedHMC/eLdgX/src/hamiltonian.jl:49
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC ~/.julia/packages/AdvancedHMC/eLdgX/src/hamiltonian.jl:49
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC ~/.julia

Chains MCMC chain (10×37×1 Array{Float64, 3}):

Iterations        = 11:1:20
Number of chains  = 1
Samples per chain = 10
Wall duration     = 747.34 seconds
Compute duration  = 747.34 seconds
parameters        = Ωm, Ωb, h, s8, ns, DESgc__0_b, DESgc__1_b, DESgc__2_b, DESgc__3_b, DESgc__4_b, DESgc__0_dz, DESgc__1_dz, DESgc__2_dz, DESgc__3_dz, DESgc__4_dz, DESwl__0_dz, DESwl__1_dz, DESwl__2_dz, DESwl__3_dz, DESwl__0_m, DESwl__1_m, DESwl__2_m, DESwl__3_m, A_IA, alpha_IA
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
   parameters      mean       std   naive_se      mcse        ess      rhat    ⋯
       Symbol   Float64   Float64    Float64   Float64    Float64   Float64    ⋯

           Ωm    0.3051    0.0429     0.0136    0.0237     2.4658    1.5162    ⋯
           Ωb    0.0401    0.0052     0.0016    0.0015     5.8

## Parallelization

Parallelization in ```Julia``` is a little bit obscure to me but this is how I do it. 
First we have to modify our script as follows:

In [ ]:
using Distributed

@everywhere begin
    using LinearAlgebra
    using Turing
    using LimberJack
    using CSV
    using YAML
    using PythonCall
    sacc = pyimport("sacc");

    println("My id is ", myid(), " and I have ", Threads.nthreads(), " threads")

    sacc_path = "../../data/FD/cls_FD_covG.fits"
    yaml_path = "../../data/DESY1/gcgc_gcwl_wlwl.yml"
    sacc_file = sacc.Sacc().load_fits(sacc_path)
    yaml_file = YAML.load_file(yaml_path)
    meta, files = make_data(sacc_file, yaml_file)

    data_vector = meta.data
    cov_tot = meta.cov
    errs = sqrt.(diag(cov_tot))
    fake_data = data_vector ./ errs
    fake_cov = Hermitian(cov_tot ./ (errs * errs'));
end 

@everywhere @model function model(data;
                                  cov=fake_cov,
                                  meta=meta, 
                                  files=files)
    #KiDS priors
    Ωm ~ Uniform(0.2, 0.6)
    Ωb ~ Uniform(0.028, 0.065)
    h ~ TruncatedNormal(72, 5, 0.64, 0.82)
    s8 ~ Uniform(0.4, 1.2)
    ns ~ Uniform(0.84, 1.1)

    DESgc__0_b ~ Uniform(0.8, 3.0)
    DESgc__1_b ~ Uniform(0.8, 3.0)
    DESgc__2_b ~ Uniform(0.8, 3.0)
    DESgc__3_b ~ Uniform(0.8, 3.0)
    DESgc__4_b ~ Uniform(0.8, 3.0)
    DESgc__0_dz ~ TruncatedNormal(0.0, 0.007, -0.2, 0.2)
    DESgc__1_dz ~ TruncatedNormal(0.0, 0.007, -0.2, 0.2)
    DESgc__2_dz ~ TruncatedNormal(0.0, 0.006, -0.2, 0.2)
    DESgc__3_dz ~ TruncatedNormal(0.0, 0.01, -0.2, 0.2)
    DESgc__4_dz ~ TruncatedNormal(0.0, 0.01, -0.2, 0.2)
    DESwl__0_dz ~ TruncatedNormal(-0.001, 0.016, -0.2, 0.2)
    DESwl__1_dz ~ TruncatedNormal(-0.019, 0.013, -0.2, 0.2)
    DESwl__2_dz ~ TruncatedNormal(0.009, 0.011, -0.2, 0.2)
    DESwl__3_dz ~ TruncatedNormal(-0.018, 0.022, -0.2, 0.2)
    DESwl__0_m ~ Normal(0.012, 0.023)
    DESwl__1_m ~ Normal(0.012, 0.023)
    DESwl__2_m ~ Normal(0.012, 0.023)
    DESwl__3_m ~ Normal(0.012, 0.023)
    A_IA ~ Uniform(-5, 5) 
    alpha_IA ~ Uniform(-5, 5)

    nuisances = Dict("DESgc__0_b" => DESgc__0_b,
                     "DESgc__1_b" => DESgc__1_b,
                     "DESgc__2_b" => DESgc__2_b,
                     "DESgc__3_b" => DESgc__3_b,
                     "DESgc__4_b" => DESgc__4_b,
                     "DESgc__0_dz" => DESgc__0_dz,
                     "DESgc__1_dz" => DESgc__1_dz,
                     "DESgc__2_dz" => DESgc__2_dz,
                     "DESgc__3_dz" => DESgc__3_dz,
                     "DESgc__4_dz" => DESgc__4_dz,

                     "A_IA" => A_IA,
                     "alpha_IA" => alpha_IA,

                     "DESwl__0_dz" => DESwl__0_dz,
                     "DESwl__1_dz" => DESwl__1_dz,
                     "DESwl__2_dz" => DESwl__2_dz,
                     "DESwl__3_dz" => DESwl__3_dz,
                     "DESwl__0_m" => DESwl__0_m,
                     "DESwl__1_m" => DESwl__1_m,
                     "DESwl__2_m" => DESwl__2_m,
                     "DESwl__3_m" => DESwl__3_m)

    cosmology = Cosmology(Ωm, Ωb, h, ns, s8,
                          tk_mode="EisHu",
                          Pk_mode="Halofit")

    theory = Theory(cosmology, meta, files; Nuisances=nuisances)
    data ~ MvNormal(theory ./ errs, cov)
end

cycles = 6
iterations = 1000
nchains = nprocs()

adaptation = 250
TAP = 0.65

stats_model = model(fake_data)
sampler = NUTS(adaptation, TAP)

println("sampling settings: ")
println("cycles ", cycles)
println("iterations ", iterations)
println("TAP ", TAP)
println("adaptation ", adaptation)
println("nchains ", nchains)

# Start sampling.
folpath = "../../chains"
folname = string("DESY1_k1k_priors_EisHu_MH")
folname = joinpath(folpath, folname)

if isdir(folname)
    fol_files = readdir(folname)
    println("Found existing file ", folname)
    if length(fol_files) != 0
        last_chain = last([file for file in fol_files if occursin("chain", file)])
        last_n = parse(Int, last_chain[7])
        println("Restarting chain")
    else
        println("Starting new chain")
        last_n = 0
    end
else
    mkdir(folname)
    println(string("Created new folder ", folname))
    last_n = 0
end

for i in (1+last_n):(cycles+last_n)
    if i == 1
        chain = sample(stats_model, sampler, MCMCDistributed(),
                       iterations, nchains, progress=true; save_state=true)
    else
        old_chain = read(joinpath(folname, string("chain_", i-1,".jls")), Chains)
        chain = sample(stats_model, sampler, MCMCDistributed(),
                       iterations, nchains, progress=true; save_state=true, resume_from=old_chain)
    end  
    write(joinpath(folname, string("chain_", i,".jls")), chain)
    CSV.write(joinpath(folname, string("chain_", i,".csv")), chain)
    CSV.write(joinpath(folname, string("summary_", i,".csv")), describe(chain)[1])
end


Here ```Distributed``` is the library managing the parallelization.
Then, each line that is evaluated by all workers must have the decorator ```@everywhere```.
Finally, in order to parallelize ```Turing``` we have to add the following inputs to ```sample(stat_model, sampler, MCMCDistributed(), iterations, nchains; progress=true, save_state=true)```

## Plotting

In order to plot the results I use ```GetDist.py``` as follows:

```python
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
import os
import getdist
from getdist import plots, MCSamples
import sacc

def add_chains(path):
    chains = []
    i = 1 
    while os.path.isfile(path+"chain_{}.csv".format(i)):
        chain = pd.read_csv(path+"chain_{}.csv".format(i))
        chains.append(chain)
        i += 1
    return pd.concat(chains)

test = add_chains("../chains/test_TAP_0.65/")

labels_dict = {'eta': '\eta',
               'l': 'l',
               'h': 'h',
               'Ωm': '\Omega_m',
               'Ωb': '\Omega_b',
               'ns': 'n_s',
               's8': '\sigma_8',
               'S8': 'S_8',
               'A_IA': 'A_{IA}',
               'alpha_IA': r'\alpha_{IA}',
               'DESgc__0_0_dz': 'dz_{DESY1gc \, 0}',
               
               'DESgc__1_0_dz': 'dz_{DESY1gc \, 1}',
               'DESgc__2_0_dz': 'dz_{DESY1gc \, 2}',
               'DESgc__3_0_dz': 'dz_{DESY1gc \, 3}',
               'DESgc__4_0_dz': 'dz_{DESY1gc \, 4}',
               
               'DESwl__0_e_dz': 'dz_{DESY1wl \, 0}',
               'DESwl__1_e_dz': 'dz_{DESY1wl \, 1}',
               'DESwl__2_e_dz': 'dz_{DESY1wl \, 2}',
               'DESwl__3_e_dz': 'dz_{DESY1wl \, 3}',
               
               'DESgc__0_0_b': 'b_{DESY1 \, 0}',
               'DESgc__1_0_b': 'b_{DESY1 \, 1}',
               'DESgc__2_0_b': 'b_{DESY1 \, 2}',
               'DESgc__3_0_b': 'b_{DESY1 \, 3}',
               'DESgc__4_0_b': 'b_{DESY1 \, 4}',
               
               'DESwl__0_e_m': 'm_{DESY1 \, 0 }',
               'DESwl__1_e_m': 'm_{DESY1 \, 1 }',
               'DESwl__2_e_m': 'm_{DESY1 \, 2 }', 
               'DESwl__3_e_m': 'm_{DESY1 \, 3 }',
               
               'eBOSS__0_0_b': 'b_{eBOSS \, 0}',
               'eBOSS__1_0_b': 'b_{eBOSS \, 1}',
               
               "DECALS__0_0_b": 'b_{DECALS \, 0}',
               "DECALS__1_0_b": 'b_{DECALS \, 1}',
               "DECALS__2_0_b": 'b_{DECALS \, 2}',
               "DECALS__3_0_b": 'b_{DECALS \, 3}',
               
               "DECALS__0_0_dz": 'dz_{DECALS \, 0}',
               "DECALS__1_0_dz": 'dz_{DECALS \, 1}',
               "DECALS__2_0_dz": 'dz_{DECALS \, 2}',
               "DECALS__3_0_dz": 'dz_{DECALS \, 3}',

               "KiDS1000__0_e_dz": 'dz_{KiDS \, 0}',
               "KiDS1000__1_e_dz": 'dz_{KiDS \, 1}',
               "KiDS1000__2_e_dz": 'dz_{KiDS \, 2}',
               "KiDS1000__3_e_dz": 'dz_{KiDS \, 3}',
               "KiDS1000__4_e_dz": 'dz_{KiDS \, 4}',
               
               "KiDS1000__0_e_m": 'm_{KiDS \, 0}',
               "KiDS1000__1_e_m": 'm_{KiDS \, 1}',
               "KiDS1000__2_e_m": 'm_{KiDS \, 2}',
               "KiDS1000__3_e_m": 'm_{KiDS \, 3}',
               "KiDS1000__4_e_m": 'm_{KiDS \, 4}',
               
               "v[1]": "v_{1}", "v[2]": "v_{2}",
               "v[3]": "v_{3}", "v[4]": "v_{4}",
               "v[5]": "v_{5}", "v[6]": "v_{6}",
               "v[7]": "v_{7}", "v[8]": "v_{8}", 
               "v[9]": "v_{9}", "v[10]": "v_{10}",
               "v[11]": "v_{11}"}

def make_chain(file, label, ranges=dict({})):
    params = np.array(list(file.keys()))
    names = []
    labels = []
    samples = []
    print()
    print(label)
    for param in params:
        print(param)
        if param in labels_dict.keys():
            print(param)
            names.append(param) 
            labels.append(labels_dict[param]) 
            samples.append(file[param])
    if ('s8' in params) & ('Ωm' in params):
        print('S8')
        names.append('S8')
        labels.append(labels_dict['S8'])
        samples.append(file['s8']*np.sqrt(file['Ωm']/0.3))

    names = np.array(names)
    labels = np.array(labels)
    samples = np.transpose(np.array(samples))
    print("========")

    return MCSamples(samples=samples, names=names, labels=labels, label=label, ranges=ranges,  
                    settings={'mult_bias_correction_order':0,'smooth_scale_2D':0.4, 'smooth_scale_1D':0.3})

test_samples = make_chain(test, "Test");

g = plots.getSubplotPlotter(subplot_size=4)
g.triangle_plot(test_samples, filled=True)
```